# Model I/O - Prompt ExampleSelector 少样本提示，示例选择器

## LengthBasedExampleSelector 按长度示例选择器

In [29]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 示例列表。
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]


# 定义如何将示例数据格式化为字符串的模板。
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# 使用LengthBasedExampleSelector选择符合长度要求的示例。
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    # 设置长度。
    max_length=8,
)

# 创建FewShotPromptTemplate，动态地根据输入选择示例并生成提示。
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)


print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: big
Output:


## MaxMarginalRelevanceExampleSelector 基于最大边际相关性的示例选择器

安装向量数据库FAISS

In [ ]:
! pip install faiss-cpu


引入类

In [30]:

from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.openai import OpenAIEmbeddings

import os

In [31]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# 创建反义词任务的示例
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

# 配置 OpenAI 服务
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  ## 设置openai的key
OPENAI_API_BASE = os.getenv('OPENAI_BASE_URL')  ## 更换为代理地址
print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("OPENAI_BASE_URL"))

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 可供选择的示例列表
    examples,
    # 用于生成语义相似性嵌入的嵌入类
    OpenAIEmbeddings(openai_api_base=OPENAI_API_BASE, openai_api_key=OPENAI_API_KEY),
    # 用于存储嵌入并进行相似性搜索的VectorStore类
    FAISS,
    # 要生成的示例数量
    k=2,
)

mmr_prompt = FewShotPromptTemplate(
    # 我们提供一个示例选择器，而不是示例
    example_selector=example_selector,
    example_prompt=example_prompt,
     prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

# 如果输入是一个情感或属性，应该首先选择相关的示例
print(mmr_prompt.format(adjective="worried"))



sk-B9B9sfxkQXtgDEAU4bkg4Iw4pwdW4budAOFVjHL8Je44foLk
https://api.fe8.cn/v1
Give the antonym of every input

Input: happy
Output: sad

Input: windy
Output: calm

Input: worried
Output:


## SemanticSimilarityExampleSelector 基于相关性的示例选择器

引入类

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.openai import OpenAIEmbeddings

import os

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n输出: {output}",
)

# 创建反义词任务的示例
examples = [
    {"input": "黑暗", "output": "光明"},
    {"input": "寒冷", "output": "温暖"},
    {"input": "干燥", "output": "湿润"},
    {"input": "安静", "output": "喧闹"},
    {"input": "平滑", "output": "粗糙"},
]

# 配置 OpenAI 服务
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  ## 设置openai的key
OPENAI_API_BASE = os.getenv('OPENAI_BASE_URL')  ## 更换为代理地址
print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("OPENAI_BASE_URL"))

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 可供选择的示例列表
    examples,
    # 用于生成语义相似性嵌入的嵌入类
    OpenAIEmbeddings(openai_api_base=OPENAI_API_BASE, openai_api_key=OPENAI_API_KEY),
    # 用于存储嵌入并进行相似性搜索的VectorStore类
    FAISS,
    # 要生成的示例数量
    k=2,
)

mmr_prompt = FewShotPromptTemplate(
    # 我们提供一个示例选择器，而不是示例
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入的反义词",
    suffix="输入: {adjective}\n输出:",
    input_variables=["adjective"],
)

# 如果输入是一个情感或属性，应该首先选择相关的示例
print(mmr_prompt.format(adjective="快乐"))



## NGramOverlapExampleSelector 按词语重叠度示例选择器

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector.ngram_overlap import NGramOverlapExampleSelector

In [ ]:
# 示例 prompt 模版
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n输出: {output}",
)

# 示例列表
examples = [
    {"input": "See Spot run.", "output": "Ver correr a Spot."},
    {"input": "My dog barks.", "output": "Mi perro ladra."},
    {"input": "Spot can run.", "output": "Spot puede correr."},
]

example_selector = NGramOverlapExampleSelector(
    # 示例列表。
    examples=examples,
    # 用于格式化示例的PromptTemplate。
    example_prompt=example_prompt,
    # 词语重叠度，0.0-1.0区间
    threshold=0.0,
)

dynamic_prompt = FewShotPromptTemplate(
    # 我们提供一个ExampleSelector而不是示例。
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="将输入的英语翻译成西班牙语",
    suffix="输入: {sentence}\n输出:",
    input_variables=["sentence"],
)

print(dynamic_prompt.format(sentence="Spot can run fast."))

